<a href="https://colab.research.google.com/github/GZembrowski/Spotify_search/blob/master/99_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ParameterGrid
import collections

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer


from sklearn import svm
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

In [0]:
X = pd.read_csv('data_with_props.csv').iloc[:,5:]
X = X.drop('type', axis=1)
X_out = pd.read_csv('albums_with_props.csv').iloc[:,7:]
X_out = X_out.drop('type', axis=1)

In [0]:
def Major(x):
    Dur_columns = [0,2,4,5,7,9,11]
    if x in Dur_columns:
        return(1)
    else:
        return(0)

    
X['majority']=X['key'].apply(Major)
X_out['majority']=X_out['key'].apply(Major)

In [0]:
columns = list(X.columns)

In [0]:
categorical = ['key', 'mode', 'time_signature', 'majority']
numerical = [e for e in columns if e not in categorical]

In [0]:
preprocess = make_column_transformer(
    (StandardScaler(),numerical),
    (OneHotEncoder(categories='auto'), categorical)
)
X = preprocess.fit_transform(X)
X_out = preprocess.transform(X_out)

In [38]:
X.shape

(7196, 34)

In [39]:
X_out.shape

(3190, 34)

In [40]:
X

array([[-0.00980163, -1.98845958, -3.25571744, ...,  0.        ,
         0.        ,  1.        ],
       [-0.27084239, -0.57740113,  0.20691385, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.75374257,  0.77051185,  0.23299349, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 1.0735175 , -0.65835686, -0.21731499, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.55143599, -1.69863805, -0.57423351, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.19903097, -0.21310032,  0.00597642, ...,  0.        ,
         1.        ,  0.        ]])

In [0]:
X_train, X_test = train_test_split(X, test_size=0.1, random_state=42)

In [0]:
def custom_scoring(clf, X_out, X_train):
    
    y_pred_train = clf.predict(X_train)
    y_pred_out = clf.predict(X_out)
    

    a = collections.Counter(y_pred_train)
    b = collections.Counter(y_pred_out)
    print(a)
    print(b)
    try:
        train = dict(a.items())[1]/(dict(a.items())[-1] + dict(a.items())[1])
        outside = dict(b.items())[-1]/(dict(b.items())[1] + dict(b.items())[-1])
        print(train)
        print(outside)
    except:
        train = 0
        
    if train > 0.9 and outside <0.8:
        result = outside
        print(result)
    else:
        result = 0

        
    return(result)

In [0]:
def fit_and_display(grid, clf, X_out, X_train):
    best_score_list = [0]
    best_params_list = []
    count=0
    for g in ParameterGrid(grid):
        count = count +1
        print(str(count) +'. ')
        clf.set_params(**g)
        clf.fit(X_train)
        score = custom_scoring(clf, X_out, X_train)
        if score > max(best_score_list):
            
            best_score_list.append(score)
            params = clf.get_params(deep=True)
            best_params_list.append(clf.get_params(params))
            print("")
            print('***********************')
            print('Best at the moment!')
            print(score)
            print(clf.get_params(deep=True))
            print('***********************')
            print("")
    return(best_score_list, best_params_list)    

In [0]:
def check_best_result(clf, best_params_list, X_train, X_test, X_out):
    clf.set_params(**best_params_list[-1])
    clf.fit(X_train)

    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    y_pred_out = clf.predict(X_out)
    print('Values for train, test and out:')
    a = (collections.Counter(y_pred_train))
    b = (collections.Counter(y_pred_test))
    c = (collections.Counter(y_pred_out))
    print('Scoring for train, test and out')
    train = dict(a.items())[1]/(dict(a.items())[-1] + dict(a.items())[1])
    test = dict(b.items())[1]/(dict(b.items())[-1] + dict(b.items())[1])
    outside = dict(c.items())[-1]/(dict(c.items())[1] + dict(c.items())[-1])
    print(train, test, outside)
    print('Parameters of the best estimator:')
    print(clf.get_params)
    print('**************************************')
    print('Best params list:')
    print(best_params_list)

In [45]:

kernels = [ 'poly', 'rbf' ]
degree = [2,4]
gammas = [0.001, 0.01, 0.1, 0.2, 0.3]
nus = [0.1, 0.15, 0.2, 0.25, 0.3]
tols = [0.001, 0.01]
grid = {'degree': degree, 'gamma' : gammas, 'kernel' : kernels, 'nu' : nus, 'tol' : tols}
clf = svm.OneClassSVM(verbose=10)

best_score_list, best_params_list = fit_and_display(grid, clf, X_out, X_train)

1. 
[LibSVM]Counter({1: 5952, -1: 524})
Counter({1: 3053, -1: 137})
0.9190858554663373
0.042946708463949844
0.042946708463949844

***********************
Best at the moment!
0.042946708463949844
{'cache_size': 200, 'coef0': 0.0, 'degree': 2, 'gamma': 0.001, 'kernel': 'poly', 'max_iter': -1, 'nu': 0.1, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': 10}
***********************

2. 
[LibSVM]Counter({-1: 3645, 1: 2831})
Counter({1: 1908, -1: 1282})
0.4371525633106856
0.40188087774294673
3. 
[LibSVM]Counter({1: 5615, -1: 861})
Counter({1: 2961, -1: 229})
0.8670475602223595
0.07178683385579937
4. 
[LibSVM]Counter({1: 5942, -1: 534})
Counter({1: 3065, -1: 125})
0.9175416924027178
0.03918495297805643
0.03918495297805643
5. 
[LibSVM]Counter({1: 5306, -1: 1170})
Counter({1: 2872, -1: 318})
0.8193329215565164
0.09968652037617555
6. 
[LibSVM]Counter({-1: 3567, 1: 2909})
Counter({1: 1923, -1: 1267})
0.44919703520691784
0.39717868338557993
7. 
[LibSVM]Counter({1: 4766, -1: 1710})


In [46]:
check_best_result(clf, best_params_list, X_train, X_test, X_out)

[LibSVM]Values for train, test and out:
Scoring for train, test and out
0.9010191476219889 0.8736111111111111 0.23887147335423198
Parameters of the best estimator:
<bound method BaseEstimator.get_params of OneClassSVM(cache_size=200, coef0=0.0, degree=2, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=10)>
**************************************
Best params list:
[{'cache_size': 200, 'coef0': 0.0, 'degree': 2, 'gamma': 0.001, 'kernel': 'poly', 'max_iter': -1, 'nu': 0.1, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': 10}, {'cache_size': 200, 'coef0': 0.0, 'degree': 2, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'nu': 0.1, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': 10}, {'cache_size': 200, 'coef0': 0.0, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'nu': 0.1, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': 10}]


In [0]:
leafs = [ 1,3,5,9,15.20,25,35,50]
neighbours = [ 1,3,5,10,15,20,25,35]
contaminations = [0.001, 0.01, 0.15,0.2,0.5]
p = [1,2]
algs = ['auto', 'ball_tree', 'kd_tree', 'brute']

grid = {'leaf_size': leafs, 'n_neighbors' : neighbours, 'contamination' : contaminations, 'algorithm' : algs, 'p': p}
clf = LocalOutlierFactor(novelty=True, n_jobs=-1)

best_score_list, best_params_list = fit_and_display(grid, clf, X_out, X_train)

1. 
Counter({1: 6476})
Counter({1: 3183, -1: 7})
2. 
Counter({1: 6476})
Counter({1: 3178, -1: 12})
3. 
Counter({1: 6476})
Counter({1: 3189, -1: 1})
4. 


In [0]:
check_best_result(clf, best_params_list, X_train, X_test, X_out)

In [0]:
estims = [ 20,50,80,120,180,400,600,900,1300,2000,3000,5000]
neighbours = [ 1,3,5,9,12,17,20,25,35,50]
contaminations = [0.001, 0.01, 0.15,0.2,0.5]
boots = [True, False]

grid = { 'n_estimators' : estims, 'contamination' : contaminations, 'bootstrap' : boots}
clf = IsolationForest(behaviour="new", n_jobs=-1, verbose=10)

best_score_list, best_params_list = fit_and_display(grid, clf, X_out, X_train)

In [0]:
check_best_result(clf, best_params_list, X_train, X_test, X_out)